In [20]:
import pandas as pd
import random
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql

In [2]:
hostname = 'localhost'
username = 'postgres'
password = '2909'
database = 'postgres'  

engine = create_engine(f'postgresql://{username}:{password}@{hostname}/{database}')

In [3]:
sql_query = "SELECT distinct report_date, sh_d, tn, vacancies FROM raw_chisl"
df = pd.read_sql(sql_query, engine)

In [4]:
df = df[df['tn'] == '0']

In [5]:
df

,report_date,sh_d,tn,vacancies
0,2022-01-01,30001,0,1.0
14,2022-01-01,30011,0,1.0
36,2022-01-01,30028,0,0.5
37,2022-01-01,30029,0,1.0
38,2022-01-01,30030,0,1.0
...,...,...,...,...
3600999,2022-12-31,39086,0,1.0
3601000,2022-12-31,39087,0,1.0
3601008,2022-12-31,39095,0,1.0
3601010,2022-12-31,39097,0,1.0


In [6]:
df['vac_clear'] = random.randint(0, 1)
df['vac_decr'] = random.randint(0, 1)
df['vac_time'] = random.randint(0, 1)

In [7]:
def fill_random(row):
    # Список столбцов для выбора
    columns = ['vac_clear', 'vac_decr', 'vac_time']
    
    # Проверка, сколько столбцов уже заполнено значением 1
    count_ones = sum(row[column] for column in columns)
    
    # Если более одного столбца имеет значение 1, выбираем один случайный столбец
    if count_ones > 1:
        # Выбираем случайный столбец, в котором значение равно 1
        random_column = np.random.choice([column for column in columns if row[column] == 1])
        # Устанавливаем значение 0 для всех столбцов, кроме выбранного
        for column in columns:
            if column != random_column:
                row[column] = 0
    else:
        # Если меньше или равно одного столбца имеет значение 1, выбираем случайный столбец для заполнения 1
        random_column = np.random.choice(columns)
        row[random_column] = 1

    return row

In [8]:
df = df.apply(fill_random, axis=1)

In [9]:
df['vac_clear'] = df['vac_clear'] * df['vacancies']
df['vac_decr'] = df['vac_decr'] * df['vacancies']
df['vac_time'] = df['vac_time'] * df['vacancies']

In [11]:
df = df[['report_date','sh_d','tn','vac_clear','vac_decr','vac_time']]

In [15]:
for i in range(len(df.count())):
    if df['vac_clear'].iloc[i]>0 and df['vac_decr'].iloc[i]>0:
        df['vac_decr'].iloc[i]=0

C:\Users\examp\AppData\Local\Temp\ipykernel_16756\36792792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vac_decr'].iloc[i]=0
C:\Users\examp\AppData\Local\Temp\ipykernel_16756\36792792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vac_decr'].iloc[i]=0
C:\Users\examp\AppData\Local\Temp\ipykernel_16756\36792792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vac_decr'].iloc[i]=0


In [16]:
for i in range(len(df.count())):
    if df['vac_clear'].iloc[i]>0 and df['vac_time'].iloc[i]>0:
        df['vac_clear'].iloc[i]=0

In [17]:
for i in range(len(df.count())):
    if df['vac_decr'].iloc[i]>0 and df['vac_time'].iloc[i]>0:
        df['vac_time'].iloc[i]=0

C:\Users\examp\AppData\Local\Temp\ipykernel_16756\227978033.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['vac_time'].iloc[i]=0
C:\Users\examp\AppData\Local\Temp\ipykernel_16756\227978033.py:3: SettingWithCopyWarning: 
A value is tr

In [22]:

# SQL-запрос для создания таблицы
create_table_query = '''
CREATE TABLE raw_vacancies (
    report_date DATE,
    sh_d VARCHAR(5),
    tn VARCHAR(5),
    vac_clear NUMERIC,
    vac_decr NUMERIC,
    vac_time NUMERIC
);
'''

# Установка соединения с базой данных
connection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cursor = connection.cursor()

# Выполнение SQL-запроса
cursor.execute(create_table_query)

# Подтверждение изменений и закрытие соединения
connection.commit()

In [23]:
# Создание подключения к базе данных
table_name = 'raw_vacancies'
df.to_sql(table_name, engine, if_exists='append', index=False)

835